In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# N-Lags model check
import yfinance as yf
# 他ファイルのインポート・リロード------------------------
import imp
import joblib
from sklearn.preprocessing import PolynomialFeatures

def data_process(data,save_path):
    count = 0
    preprocess_data = pd.DataFrame()

    # # 作業ディレクトリの移動と確認
    os.chdir('/Users/suzukiryotaro/Desktop/python_stock/machine_learning/data_process/')
    # 他ファイルのインポート・リロード
    import data_process
    imp.reload(data_process)
    for  file in data['symbol'] :
        count = count +1
        print(count)
        print(file)
#         try:
        df = yf.download(file, start='2014-01-01')
        df = data_process.create_data(df)
        df = df['2016-01-06':'2020-12-31']
        preprocess_data = preprocess_data.append(df)

#         except: 
#             print('errror')
#             continue
    # drop row contains NaN
    preprocess_data.dropna(inplace=True)
    preprocess_data.to_csv(save_path)
    print('saved')

    return preprocess_data


#全時系列データの呼び出し
model_stock = 'nasdaq100'
path = '../../../stock_data/stock_code/{}.csv'.format(model_stock)
path = '/Users/suzukiryotaro/Desktop/python_stock/stock_data/stock_code/sandp500.csv'
data = pd.read_csv(path)
# print(data)

#データの前処理　※一度やったら飛ばす
save_path = '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/genetic_argo/preprocss_data.csv'
df = data_process(data,save_path)

df = pd.read_csv(save_path, index_col=0)
#選択する特徴量
cols =  ['return','sma210'

          ,'ratio_sma75_30', 'ratio_sma75_150', 'ratio_sma105_30', 
          'ratio_sma105_150', 'ratio_sma135_150'
          ,'adosc', 'sma10', 
        'adosc-SG' ,'typical-price',"ema270"
      ,'Highest121','Highest45,81days_ago',

'today_by_sma30ratio','today_by_sma45ratio','today_by_sma60ratio','today_by_sma120ratio',
         
'Adjclose_today_40daysago_ratio','Adjclose_today_121daysago_ratio','high_low_ratio_30_sma'

,'today_by_high_low_ratio_20_sma_ratio','today_by_high_low_ratio_40_sma_ratio','today_by_high_low_ratio_90_sma_ratio','today_by_high_low_ratio_120_sma_ratio'

,'high_low_ratio_20_sma_ratio','high_low_ratio_40_sma_ratio','high_low_ratio_60_sma_ratio','Adjclose_today_15daysago_ratio','high_low_ratio_30_sma'

,'today_by_sma120ratio','high_low_ratio','today_by_sma60ratio','today_by_sma120ratio',"ema270",

'adosc-SG','Highest121','Highest161','today_by_sma60ratio','Highest45,81days_ago',

'Highest81','Highest45,150days_ago','Highest15,30days_ago'

,'Highest121,60days_ago',

'Highest15','adosc_ratio','Highest45','Highest61','Highest15,30days_ago' ,'Highest15,60days_ago'

,'Highest61','adosc_ratio','Highest45', 'ratio_sma15_30','Highest15,30days_ago'

"NDM",'PDI','NDI','DX','ADX','ADXR','Standard_deviation_normalization10'

,'aroon_up14','aroon_up20','aroon_down14','aroon_down20','aroon_ratio14','aroon_ratio20',"TrueRange","PDM"

,"ema30","ema90","ema150","ema200", 'OBV_ratio','OBV_EMA_ratio'
         
'mtm10','A/D','A/D_EMA','A/D_ratio','ADOSC','A/D_EMA_ratio','OBV','OBV_EMA','OBV_ratio'

, 'rsi', 'rsi-FP', 'ar26', 'br26' ,'bias20', 'cci-SG' 'roc-SG','macd-histogram', 'roc-FP

,'sma30','sma270','Highest121'
,'adosc_ratio', 'Volume', 'amount','sma10-FP','macd', 'macd-SG','Standard_deviation_normalization40'

'Highest45','Highest81,30days_ago','Highest81','Standard_deviation_normalization30','Highest121','Highest61','Highest121,30days_ago','sma150', 'ratio_sma135_30'


          ]

X = df[cols].copy()
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y = X.pop('return')

# 多項式特徴量を追加
poly = PolynomialFeatures(2)
poly.fit(X)
X_poly = pd.DataFrame(poly.transform(X), columns=poly.get_feature_names(input_features=X.columns))

# 特徴量が多いパターンを検証するために、PolynomialFeaturesを使って特徴量を増やしたものがX_poly_sc
X_sc = pd.DataFrame(sc.fit_transform(X), columns=X.columns)
X_poly_sc = pd.DataFrame(sc.fit_transform(X_poly), columns=X_poly.columns)




def evalIndividual(individual):
    """ 個体評価のための関数
        Prameter:
        individual: 要素が0or1のリスト  長さは入力変数の数と同じ
        Return:
        n_features: 使った特徴量の数
        score: モデルの精度
    """
    n_features = sum(individual)

    if n_features == 0:
        return 9999, -9999

    X = X_poly_sc.iloc[:, [bool(val) for val in individual]]
    
#     scores = []
#     for _ in range(30):
#         X_train, X_test, y_train, y_test = train_test_split(X_temp, y, test_size=0.4) 
#         model = RidgeCV()
#         model.fit(X_train, y_train)
#         scores.append(model.score(X_test, y_test))

#     score = np.array(scores).mean()
    
    
    X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.15, random_state=11)
    X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.3, random_state=11)
    # train用・valid用のデータセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val) 

    params = {'metric': 'rmse',
              'max_depth' : 20,
              'num_leaves': 23, # 決定木の複雑度を調整（初期値31）'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
              'boosting_type': 'gbdt', # 勾配ブースティング
              'objective': 'regression', # 目的 : 回帰  
              'num_leaves': 23, # 決定木の複雑度を調整（初期値31）
              'min_data_in_leaf': 15, # データの最小数（初期値20） 
              "learning_rate": 0.025400643228660445,
              'gamma':0.20387750446308275, 
              }

    model = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=10000,
                    early_stopping_rounds=100,
                    verbose_eval=50
                    )

    return n_features, score



creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))   
creator.create("Individual", list, fitness=creator.Fitness)


toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_bool, X_poly_sc.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", evalIndividual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

"""遺伝的アルゴリズム設定
50世代まで
1世代の個体数300
次世代に引き継ぐ個体数100
交叉確率0.7
突然変異確率0.1
"""
NGEN = 50
MU =　300
LAMBDA = 100
CXPB = 0.7
MUTPB = 0.1

pop = toolbox.population(n=MU)
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

pop, log = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB,
                                     MUTPB, NGEN, stats, halloffame=hof)






joblib.dump(model, '/Users/suzukiryotaro/Desktop/python_stock/machine_learning/ansamble/lightgbm/train/lightgbm0706.joblib') 


# 特徴量重要度の算出 (データフレームで取得)
# .drop('return',axis=1)
cols = list(X.columns)         # 特徴量名のリスト(目的変数CRIM以外)
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)



# 特徴量重要度を棒グラフでプロットする関数 
def plot_feature_importance(df): 
    n_features = len(df)                              # 特徴量数(説明変数の個数) 
    df_plot = df.sort_values('importance')            # df_importanceをプロット用に特徴量重要度を昇順ソート 
    f_importance_plot = df_plot['importance'].values  # 特徴量重要度の取得 
    plt.barh(range(n_features), f_importance_plot, align='center') 
    cols_plot = df_plot['feature'].values             # 特徴量の取得 
    plt.yticks(np.arange(n_features), cols_plot)      # x軸,y軸の値の設定
    plt.xlabel('Feature importance')                  # x軸のタイトル
    plt.ylabel('Feature')                             # y軸のタイトル
# 特徴量重要度の可視化
plot_feature_importance(df_importance)


'''
特徴量リスト

#ストック
#
# 'Highest14',        'today_by_sma30ratio','today_by_sma45ratio','today_by_sma60ratio','today_by_sma120ratio'
'Highest81','Highest45,150days_ago','sma90',

    


脱落11

'Highest45,150days_ago','Highest15,30days_ago'

脱落１０
,'Highest121,60days_ago'
脱落9
'Highest15','adosc_ratio','Highest45','Highest61','Highest15,30days_ago' ,'Highest15,60days_ago'
脱落8
,'Highest61','ratio_sma15_30','Highest15,30days_ago'
脱落7
"NDM",'PDI','NDI','DX','ADX','ADXR','Standard_deviation_normalization10'
脱落6
,'aroon_up14','aroon_up20','aroon_down14','aroon_down20','aroon_ratio14','aroon_ratio20',"TrueRange","PDM"
脱落5
,"ema30","ema90","ema150","ema200", 'OBV_ratio','OBV_EMA_ratio'
脱落4
', 'mtm10','A/D','A/D_EMA','A/D_ratio','ADOSC','A/D_EMA_ratio','OBV','OBV_EMA','OBV_ratio'
#脱落3
, 'rsi', 'rsi-FP', 'ar26', 'br26' ,'bias20',, 'cci-SG' 'roc-SG','macd-histogram', 'roc-FP
#脱落2
,'sma30',,'sma270','Highest121'
,'adosc_ratio', 'Volume', 'amount','sma10-FP','macd', 'macd-SG',,'Standard_deviation_normalization40'
#脱落組
'Highest45','Highest81,30days_ago','Highest81','Standard_deviation_normalization30','Highest121','Highest61','Highest121,30days_ago','sma150', 'ratio_sma135_30'

'''